<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/Assembly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autorch > log.txt
!pip install optuna > log.txt

In [2]:
import joblib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import autorch
from autorch.function import sp2wt
import optuna
from tqdm import tqdm_notebook as tqdm

# LOAD DATA

In [4]:
icg_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl')
c620_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl')
c660_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl')
t651_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/t651_col_names.pkl')
c670_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl')

# 有共同index的部分

In [5]:
icg_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/icg_train.csv',index_col=0).dropna(axis=0)
c620_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0).dropna(axis=0)
c660_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_train.csv',index_col=0).dropna(axis=0)
c670_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c670_train.csv',index_col=0).dropna(axis=0)
t651_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/t651_train.csv',index_col=0).dropna(axis=0)
allidx = list(set(icg_df.index)&
      set(c620_df.index)&
      set(c660_df.index)&
      set(c670_df.index)&
      set(t651_df.index))
len(allidx)

1295

# INPUT端

In [6]:
# icg
icg_input = icg_df.loc[allidx,icg_c['x']]
icg_input = icg_input.join(c620_df.loc[allidx,'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'])
icg_input = icg_input.join(c660_df.loc[allidx,'Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'])
icg_input = icg_input.join(c620_df.loc[allidx].filter(regex='Receiver Temp'))

# c620
c620_feed = c620_df.loc[allidx,c620_c['x41']]

# t651
t651_feed = t651_df.loc[allidx,t651_c['x41']]

# OUTPUT端

In [7]:
c620_op = c620_df.loc[allidx,c620_c['density']+c620_c['yRefluxRate']+c620_c['yHeatDuty']+c620_c['yControl']]
c620_wt = c620_df.loc[allidx,c620_c['vent_gas_x']+c620_c['distillate_x']+c620_c['sidedraw_x']+c620_c['bottoms_x']]
c660_op = c660_df.loc[allidx,c660_c['density']+c660_c['yRefluxRate']+c660_c['yHeatDuty']+c660_c['yControl']]
c660_wt = c660_df.loc[allidx,c660_c['vent_gas_x']+c660_c['distillate_x']+c660_c['sidedraw_x']+c660_c['bottoms_x']]
c670_op = c670_df.loc[allidx,c670_c['density']+c670_c['yRefluxRate']+c670_c['yHeatDuty']+c670_c['yControl']]
c670_wt = c670_df.loc[allidx,c670_c['distillate_x']+c670_c['bottoms_x']]

# CONFIG

In [8]:
config = {
      # c620
      'c620_G':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_G.pkl',
      'c620_F':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_F.pkl',
      'c620_op_min':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_op_min.pkl',
      'c620_op_max':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_op_max.pkl',

      # c660
      'c660_G':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660_G.pkl',
      'c660_F':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660_F.pkl',
      'c660_op_min':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_op_min.pkl',
      'c660_op_max':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_op_max.pkl',

      # c670
      'c670_M':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c670.pkl',
      
      # col_names
      'icg_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl',
      'c620_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl',
      'c660_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl',
      'c670_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl',
      
      # Special column (0.9999 & 0.0001)
      'index_9999_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_9999.pkl',
      'index_0001_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_0001.pkl',

      # sp
      'c620_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_wt_always_same_split_factor_dict.pkl',
      'c660_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_wt_always_same_split_factor_dict.pkl',
      'c670_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c670_wt_always_same_split_factor_dict.pkl',

      'op_fix_model':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/fix_model.pkl',
      
          }

# 輸入與輸出的對應關係

In [9]:
# 對應關係1
a = icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]
b = c620_df['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']
a.join(b).sample(5)

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
023-005,90.0,90.000000
079-020,70.0,70.000008
103-020,70.0,70.000000
074-026,70.0,70.000008
119-008,90.0,90.000000


In [10]:
#對應關係2
a = icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
b = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1)*10000
b.name = 'c660_wt_NA_in_Benzene_ppmw'
a.join(b).sample(5)

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,c660_wt_NA_in_Benzene_ppmw
132-011,980.0,980.007580
019-011,980.0,980.002658
035-020,980.0,980.004718
097-008,920.0,920.004338
120-023,860.0,859.999895


In [11]:
#對應關係3
a = icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]
b = c660_df['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']*10000
a.join(b).sample(5)

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%
011-026,9.999987,9.999987
081-023,10.000041,10.000041
013-017,5.000000,5.000000
086-011,9.999999,9.999999
141-020,9.999985,9.999985


In [12]:
class AllSystem(object):
  def __init__(self,config):
    
    # C620 部份模組
    self.c620_G = joblib.load(config['c620_G'])
    self.c620_F = joblib.load(config['c620_F'])
    self.c620_op_min = joblib.load(config['c620_op_min'])
    self.c620_op_max = joblib.load(config['c620_op_max'])
    
    # C660 部份模組
    self.c660_G = joblib.load(config['c660_G'])
    self.c660_F = joblib.load(config['c660_F'])
    self.c660_op_min = joblib.load(config['c660_op_min'])
    self.c660_op_max = joblib.load(config['c660_op_max'])

    # C670 部份模組
    self.c670_M = joblib.load(config['c670_M'])
    
    # 用來修正現場儀表的誤差
    self.op_fix_model = joblib.load(config['op_fix_model'])

    # 欄位名稱列表
    self.icg_col = joblib.load(config['icg_col_path'])
    self.c620_col = joblib.load(config['c620_col_path'])
    self.c660_col = joblib.load(config['c660_col_path'])
    self.c670_col = joblib.load(config['c670_col_path'])
    
    # 其他資訊
    self.c620_wt_always_same_split_factor_dict = joblib.load(config['c620_wt_always_same_split_factor_dict'])
    self.c660_wt_always_same_split_factor_dict = joblib.load(config['c660_wt_always_same_split_factor_dict'])
    self.c670_wt_always_same_split_factor_dict = joblib.load(config['c670_wt_always_same_split_factor_dict'])
    self.index_9999 = joblib.load(config['index_9999_path'])
    self.index_0001 = joblib.load(config['index_0001_path'])

    # 由廠區方提供的平均密度用來將體積流量轉成重量流量
    self.V615_density = 0.8626
    self.C820_density = 0.8731
    self.T651_density = 0.8749

    # 操作欄位名稱列表
    self.c620_op_col = self.c620_G.y_col # G系列模型本身就是預測操作條件
    self.c660_op_col = self.c660_G.y_col # G系列模型本身就是預測操作條件
    self.c670_op_col = self.c670_M.y_col[41*2:] #前面82個是分離係數後面都是操作條件

    # 廠區通常調整欄位列表
    self.c620_op_col_can_change = ['Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC','Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC']
    self.c660_op_col_can_change = ['Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC','Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC']
  
  def inference(self,icg_input,c620_feed,t651_feed,
                real_data_mode = False #修正現場儀表誤差用
                ):
    
    # 紀錄樣本index
    idx = icg_input.index
    
    # 計算c620_case
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    
    # 預測c620操作條件和分離係數
    c620_op = self.c620_G.predict(c620_case.join(c620_feed))
    c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op))
    
    # 計算c620輸出組成
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    #計算c660_feed
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1)
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1)
    V615_Btm_ton = V615_Btm_m3*self.V615_density
    C820_Dist_ton = C820_Dist_m3*self.C820_density
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True)
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True)
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1)
    c660_mf = t651_mf + c620_mf_side
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])
    
    # 計算c660_case
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case'])
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values
    c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values
    
    # 預測c660操作條件和分離係數
    c660_op = self.c660_G.predict(c660_case.join(c660_feed))
    c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op))
    
    # 計算c660輸出組成
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    # 計算c670_feed
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True)
    c670_mf = c620_mf_bot + c660_mf_bot
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])
    
    # 計算c670_upper_bf
    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf'])
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[list(set(self.index_9999)&set(upper_bf.columns))] = 0.9999
    upper_bf[list(set(self.index_0001)&set(upper_bf.columns))] = 0.0001
    
    # 直接預測分離係數和操作條件
    c670_output = self.c670_M.predict(c670_feed.join(upper_bf))
    c670_sp,c670_op = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]    
    
    # 計算輸出組成
    s1,s2 = c670_sp[self.c670_col['distillate_sf']].values,c670_sp[self.c670_col['bottoms_sf']].values
    w1,w2 = sp2wt(c670_feed,s1),sp2wt(c670_feed,s2)
    c670_wt = pd.DataFrame(np.hstack((w1,w2)),index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    
    # 是否修正操作條件 for 現場數據
    if real_data_mode == False:
      return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op
    
    if real_data_mode == True:
      # 有些欄位現場數據沒有
      c620_op_col = c620_op.drop(['Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      c660_op_col = c660_op.drop(['Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      c670_op_col = c670_op.drop(['Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      
      # 經過修正模組修正op
      op_pred = self.op_fix_model(torch.cat((
        torch.FloatTensor(c620_op[c620_op_col].values),
        torch.FloatTensor(c660_op[c660_op_col].values),
        torch.FloatTensor(c670_op[c670_op_col].values)),dim=1))
      op_pred = pd.DataFrame(op_pred.detach().numpy(),index=idx)
      
      # 新的op
      new_c620_op = op_pred.iloc[:,:8]
      new_c660_op = op_pred.iloc[:,8:16]
      new_c670_op = op_pred.iloc[:,-5:]
      new_c620_op.columns = c620_op_col
      new_c660_op.columns = c660_op_col
      new_c670_op.columns = c670_op_col
      
      # 更新op
      c620_op.update(new_c620_op)
      c660_op.update(new_c660_op)
      c670_op.update(new_c670_op)
      
      return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op
  
  def recommend(self,icg_input,c620_feed,t651_feed,
                search_iteration = 300, # cma-es優化搜索次數
                real_data_mode = False, # 如果打開這個功能則會把操作條件在經過一個修正模組來配合現場儀表偏移
                auto_set_recommended_value = [70,980,10], #自動設定推薦值
                only_tune_temp = False, # 是否只調溫度,如果其他也可以調可能可以比較快找到理想的解,否則可能找不到理想的解
                ):
    
    self.only_tune_temp = only_tune_temp
    # 先用試算模式試算一遍
    idx = icg_input.index
    c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = self.inference(icg_input,c620_feed,t651_feed,real_data_mode=real_data_mode) 

    # 紀錄該筆樣本原始的Benzene in C620 Sidedraw_wt%
    original_Benzene_in_C620_Sidedraw_wt = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values[0]
    
    # 是否自動設定推薦值
    if len(auto_set_recommended_value) == 3:
      icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'] = auto_set_recommended_value[0]
      icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] = auto_set_recommended_value[1]
      icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = auto_set_recommended_value[2]
      print(f'系統已經自動設定推薦值[{auto_set_recommended_value[0]},{auto_set_recommended_value[1]},{auto_set_recommended_value[2]}]')
    
    # 根據推薦的Benzene in C620 Sidedraw_wt%和原始的Benzene in C620 Sidedraw_wt%去比較來判斷說溫度要往上調還是往下調
    if len(auto_set_recommended_value) == 3:
      # 代表現在工程師想要把原本的Benzene_in_C620_Sidedraw_wt往上拉所以溫度調幅應該往下
      if original_Benzene_in_C620_Sidedraw_wt < auto_set_recommended_value[0]:
        self.temp_adjust_direction = 'down'
      # 代表現在工程師想要把原本的Benzene_in_C620_Sidedraw_wt往下降所以溫度調幅應該往上
      if original_Benzene_in_C620_Sidedraw_wt > auto_set_recommended_value[0]:
        self.temp_adjust_direction = 'up'
      # 如果兩者一樣就不特別規定調幅要往上還往下,任意即可
      if original_Benzene_in_C620_Sidedraw_wt == auto_set_recommended_value[0]:
        self.temp_adjust_direction = 'arbitrary'
    
    # c620 case設置
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    
    # c660 case設置
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case'])
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values
    c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values
    
    # cma-es 初始值設置 操作條件初始值來自於試算值
    x0 = {}
    for name in self.c620_op_col:
      x0[name] = c620_op[name].values[0]
    for name in self.c660_op_col:
      x0[name] = c660_op[name].values[0]
    
    # 這一樣Distillate Rate_m3/hr的初始值來自於使用者輸入
    x0['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values[0]
    
    # 建立 cma-es study 並賦予初始值X0
    sampler = optuna.samplers.CmaEsSampler(x0=x0)
    study = optuna.create_study(sampler=sampler)
    
    # cma-es 搜索階段
    history = {}
    for i in tqdm(range(search_iteration)):
      trial = study.ask()
      
      # 1.在c620_op的向量空間作採樣
      c620_op_opt_dict = {}
      minimum_amplitude = 0.1
      for name in self.c620_op_col:
        # 如果這些欄位屬於可以調的 例如廠區通常調整溫度
        if name in self.c620_op_col_can_change:
          if self.temp_adjust_direction == 'down':
            try:
              c620_op_opt_dict[name] = trial.suggest_uniform(name,self.c620_op_min[name],c620_op[name].values[0]-minimum_amplitude) #最少應該降minimum_amplitude度
            except:
              print('溫度已經低於訓練數據的最小值不能再低了')
              c620_op_opt_dict[name] = self.c620_op_min[name] # 所以取最小值
          if self.temp_adjust_direction == 'up':
            try:
              c620_op_opt_dict[name] = trial.suggest_uniform(name,c620_op[name].values[0]+minimum_amplitude,self.c620_op_max[name]) #最少應該增minimum_amplitude度
            except:
              print('溫度已經低於訓練數據的最大值不能再高了')
              c620_op_opt_dict[name] = self.c620_op_max[name] # 所以取最大值
          if self.temp_adjust_direction == 'arbitrary':
            c620_op_opt_dict[name] = trial.suggest_uniform(name,self.c620_op_min[name],self.c620_op_max[name]) #任意調
        # 比較不能調的,看要不要調,可以調的話找到理想解的可能性會比較高
        else:
          if self.only_tune_temp == True:
            c620_op_opt_dict[name] = c620_op[name].values[0] 
          if self.only_tune_temp == False:
            c620_op_opt_dict[name] = trial.suggest_uniform(name,self.c620_op_min[name],self.c620_op_max[name]) #任意調
      c620_op_opt = pd.DataFrame(c620_op_opt_dict,index=idx)
      
      # 2.在c660_op的向量空間作採樣,但由於c660目前沒有跟c620一樣的苯和溫度呈現負相關這樣的限制,暫時沒有特別去設計調幅方向
      c660_op_opt_dict = {}
      for name in self.c660_op_col:
        if name in self.c660_op_col_can_change:
          c660_op_opt_dict[name] = trial.suggest_uniform(name,self.c660_op_min[name],self.c660_op_max[name])
        
        # 比較不能調的,看要不要調,可以調的話找到理想解的可能性會比較高
        else:
          if self.only_tune_temp == True:
            c660_op_opt_dict[name] = c660_op[name].values[0] 
          if self.only_tune_temp == False:
            c660_op_opt_dict[name] = trial.suggest_uniform(name,self.c660_op_min[name],self.c660_op_max[name]) #任意調
      c660_op_opt = pd.DataFrame(c660_op_opt_dict,index=idx)
      
      # 3.在Operation_Specifications_Spec 2 : Distillate Rate_m3/hr的向量空間做採樣,並將採樣結果代入c620_case
      c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = trial.suggest_float(
          'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr',0,10.25)
      
      # 計算c620輸出wt%
      c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op_opt))
      s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
      w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
      wt = np.hstack((w1,w2,w3,w4))
      c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
      
      # 計算c660輸入_wt%
      V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1) # V615體積流量
      C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1) # C820體積流量
      V615_Btm_ton = V615_Btm_m3*self.V615_density # V615體積流量轉重量流量
      C820_Dist_ton = C820_Dist_m3*self.C820_density # C820體積流量轉重量流量
      c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton # V615和C820兩道油相加得到c620總入料的"質量流量"
      c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True) # 根據公式換算出c620_side流量
      c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True) # 根據公式換算出c620_bott流量
      t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1) # t651質量流量轉成重量流量
      c660_mf = t651_mf + c620_mf_side # t651和c620_side兩道油的質量流量相加得到c660入料質量流量
      t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf # t651和c620_side兩道油對c660總入料質量流量的占比百分比
      c660_feed = c620_wt[self.c620_col['sidedraw_x']].values * c620_mf_side_p + t651_feed.values * t651_mf_p #根據公式換算c660混合入料組成
      c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41']) # 將c660入料組成轉換成dataframe格式
      
      # 計算c660輸出wt%
      c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op_opt))
      s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
      w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
      wt = np.hstack((w1,w2,w3,w4))
      c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
      
      # 計算損失
      input_bzinside = c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'].values[0]
      input_nainbz = c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'].values[0]
      input_tol = c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values[0]
      input_dist_rate = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values[0]
      
      output_bzinside = c620_wt['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%'].values[0]
      output_nainbz = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,[1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]].sum(axis=1).values[0]*10000
      output_tol = c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000
      output_dist_rate = c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values[0]
      
      bzinside_loss = abs(input_bzinside - output_bzinside) / input_bzinside #絕對百分比誤差
      nainbz_loss = abs(input_nainbz - output_nainbz) / input_nainbz #絕對百分比誤差
      tol_loss = abs(input_tol - output_tol) / input_tol #絕對百分比誤差
      distrate_loss = max(output_dist_rate - input_dist_rate,0) # distrate根據廠區說法愈小愈好,因此如果採樣出的如果採樣出的dist_rate大於input_dist_rate就會有loss,否則為0
      total_loss = bzinside_loss + nainbz_loss + tol_loss + distrate_loss #總損失
      
      # 把總損失告訴study供下一次採樣的依據
      study.tell(trial,total_loss)

      # 如果滿足以下條件就算提早成功
      if (bzinside_loss<=0.02) and (nainbz_loss<=0.05) and (tol_loss<=0.1) and (distrate_loss==0):
        print('Congratulations Early Success find optimal op')
        break
    
    # cma-es搜索迴圈跑完或是提早結束,調出study.best_params並製作成best_params_df
    best_params_df = pd.DataFrame(
        study.best_params,
        index = idx,
        columns = self.c620_op_col + self.c660_op_col + ['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'])
    
    # 優化過的c620_op,c660_op,c620_case(dist_rate)
    c620_op_opt = c620_op.drop(self.c620_op_col_can_change,axis=1).join(best_params_df[self.c620_op_col][self.c620_op_col_can_change])
    c660_op_opt = c660_op.drop(self.c660_op_col_can_change,axis=1).join(best_params_df[self.c660_op_col][self.c660_op_col_can_change])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = best_params_df['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr']
    
    # 計算c620輸出wt%
    c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op_opt))
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    # 計算c660_feed_wt%
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1) #體積流量
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1) #體積流量
    V615_Btm_ton = V615_Btm_m3*self.V615_density # 體積流量轉重量流量
    C820_Dist_ton = C820_Dist_m3*self.C820_density # 體積流量轉重量流量
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton # 兩股相加得到c620總入料重量流量
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True) # c620_side 流量
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True) # c620_bott 流量
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1) # t651重量流量
    c660_mf = t651_mf + c620_mf_side # 兩股相加得到c660入料重量流量
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf # 兩股油源的占比百分比
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p #計算c660入料組成
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41']) #計算c660入料組成(dataframe格式)

    # 計算c660輸出wt%
    c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op_opt))
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    # 計算損失
    input_bzinside = c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'].values[0] 
    input_nainbz = c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'].values[0] 
    input_tol = c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values[0]
    input_dist_rate = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values[0]
    
    output_bzinside = c620_wt['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%'].values[0] 
    output_nainbz = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,[1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]].sum(axis=1).values[0]*10000 
    output_tol = c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000
    output_dist_rate = c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values[0]
    
    bzinside_loss = abs(input_bzinside - output_bzinside) / input_bzinside
    nainbz_loss = abs(input_nainbz - output_nainbz) / input_nainbz
    tol_loss = abs(input_tol - output_tol) / input_tol
    distrate_loss = max(output_dist_rate - input_dist_rate,0)
    
    # 打印優化結果
    print('bzinside_loss:',bzinside_loss)
    print('nainbz_loss:',nainbz_loss)
    print('tol_loss:',tol_loss)
    print('distrate_loss:',distrate_loss)

    # c670 部份
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True) # c660_bot 重量流量
    c670_mf = c620_mf_bot + c660_mf_bot # c620_bot 重量流量 加上 c660_bot 重量流量 得到c670總入料重量流量
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf # 兩股油源的百分比占比
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p # c670入料組成
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])
    
    # c670 upper_bf 計算
    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf']) 
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[list(set(self.index_9999)&set(upper_bf.columns))] = 0.9999
    upper_bf[list(set(self.index_0001)&set(upper_bf.columns))] = 0.0001
    
    # c670因為沒有指定wt數值要多少所以直接預測操作條件和分離係數即可
    c670_output = self.c670_M.predict(c670_feed.join(upper_bf))
    c670_sp,c670_op_opt = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]    
    
    # 計算c670_wt
    s1 = c670_sp[self.c670_col['distillate_sf']].values
    s2 = c670_sp[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    
    # 是否修正操作條件 for 現場數據
    if real_data_mode == False:
      return c620_wt,c620_op_opt,c660_wt,c660_op_opt,c670_wt,c670_op_opt,bzinside_loss,nainbz_loss,tol_loss
    
    if real_data_mode == True:
      # 有些欄位現場數據沒有 因此drop掉
      c620_op_col = c620_op.drop(['Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      c660_op_col = c660_op.drop(['Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      c670_op_col = c670_op.drop(['Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      
      # 經過修正模組修正op
      op_pred = self.op_fix_model(torch.cat((
        torch.FloatTensor(c620_op_opt[c620_op_col].values),
        torch.FloatTensor(c660_op_opt[c660_op_col].values),
        torch.FloatTensor(c670_op_opt[c670_op_col].values)),dim=1))  
      op_pred = pd.DataFrame(op_pred.detach().numpy(),index=idx)
      
      # 新的op
      new_c620_op = op_pred.iloc[:,:8]
      new_c660_op = op_pred.iloc[:,8:16]
      new_c670_op = op_pred.iloc[:,-5:]
      new_c620_op.columns = c620_op_col
      new_c660_op.columns = c660_op_col
      new_c670_op.columns = c670_op_col
      
      # 更新op
      c620_op_opt.update(new_c620_op)
      c660_op_opt.update(new_c660_op)
      c670_op_opt.update(new_c670_op)
      
      return c620_wt,c620_op_opt,c660_wt,c660_op_opt,c670_wt,c670_op_opt,bz_error,nainbz_error,tol_error

# 試算模式測試 觀察重點 準

In [13]:
f = AllSystem(config)
idx = np.random.choice(allidx,size=100,replace=False,p=None)
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f.inference(icg_input.loc[idx],c620_feed.loc[idx],t651_feed.loc[idx])

In [14]:
f.c670_M.show_metrics(c620_wt.loc[idx],c620_wt_,e=2e-2)

,R2,MSE,MAPE
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.754057,0.00126601,1.91402
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.745575,0.397749,1.44179
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.314079,0.0179781,0.278705
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.706415,0.187528,3.9995
...,...,...,...
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999354,1.88027e-06,0.297334
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999343,3.46589e-07,0.297333
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [15]:
f.c670_M.show_metrics(c620_op.loc[idx],c620_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.999821,2.68336e-10,0.00150684
Density_Vent Gas Production Rate and Composition,0.901638,8.26239e-07,0.164259
Density_Distillate Production Rate and Composition,0.752615,1.46335e-05,0.422551
Density_Sidedraw Production Rate and Composition,0.994815,5.59164e-09,0.00626787
Density_Bottoms Production Rate and Composition,0.996838,5.39133e-11,0.000713593
Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,0.988898,1.21396,0.66231
Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.99105,0.00905123,0.543005
Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.989654,0.0147432,0.607468
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,0.987827,0.0158201,0.0556967
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,0.990621,0.0073519,0.0356008


In [16]:
f.c670_M.show_metrics(c660_wt.loc[idx],c660_wt_,e=2e-2)

,R2,MSE,MAPE
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.850502,0.0296512,11.3149
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.91534,3.62668,6.89606
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.646236,2.18207,5.22386
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.926688,0.974215,11.7783
...,...,...,...
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.970376,3.21376e-25,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.971701,3.74067e-31,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [20]:
f.c670_M.show_metrics(c660_op.loc[idx],c660_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.989302,9.23344e-09,0.0083903
Density_Vent Gas Production Rate and Composition,0.948664,1.83715e-05,0.677648
Density_Distillate (Benzene Drag) Production Rate and Composition,0.955949,3.76232e-06,0.186739
Density_Sidedraw (Benzene )Production Rate and Composition,0.943916,3.1405e-12,0.000160082
Density_Bottoms Production Rate and Composition,0.993332,5.10393e-10,0.00203276
Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,0.968778,25.6778,3.10928
Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.966088,0.210499,2.83437
Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.968361,0.208485,2.95811
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,0.970367,0.00124529,0.0335929
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC,0.955867,0.0337064,0.151731


In [21]:
f.c670_M.show_metrics(c670_wt.loc[idx],c670_wt_,e=2e-2)

,R2,MSE,MAPE
Toluene Column C670 Operation_Distillate Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Methane_wt%,0,1.58458e-10,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Ethane_wt%,0,9.37471e-08,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Propane_wt%,0,8.02615e-08,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Butane_wt%,0,5.81595e-47,NaN
...,...,...,...
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999999,2.8229e-08,0.0192961
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999999,5.11633e-09,0.0193115
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [22]:
f.c670_M.show_metrics(c670_op.loc[idx],c670_op_,e=2e-2)

,R2,MSE,MAPE
Density_Distillate Production Rate and Composition,0.995709,3.67552e-12,0.000167189
Density_Bottoms Production Rate and Composition,0.995428,8.25372e-10,0.0024835
Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,0.980356,10.6337,1.06152
Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.985758,0.115808,0.957873
Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.985067,0.115208,0.942308
Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,0.950687,0.00229593,0.0207367
Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC,0.986786,0.0333463,0.0610766
AVG,0.982827,1.55719,0.435166


# 試算模式 除了準還要 確認三項條件有滿足

In [23]:
a = icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]
b = c620_wt[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]
a.join(b)

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
027-023,70.0,70.000000
029-002,90.0,90.000000
088-023,70.0,70.000015
112-014,80.0,80.000000
080-017,80.0,79.999992
...,...,...
069-017,80.0,79.999992
077-008,90.0,89.999992
084-011,80.0,79.999992
123-005,90.0,90.000000


In [24]:
a = icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]
b = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,[1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] ].sum(axis=1)*10000
b.name = 'c660_wt NA in NA in Benzene_ppmw'
a.join(b)

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,c660_wt NA in NA in Benzene_ppmw
027-023,950.0,950.000152
029-002,980.0,979.995359
088-023,950.0,949.998613
112-014,950.0,949.999099
080-017,920.0,920.002867
...,...,...
069-017,920.0,920.159994
077-008,920.0,919.996985
084-011,890.0,889.993016
123-005,950.0,950.000608


In [25]:
a = icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]
b = c660_wt[['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']]*10000
a.join(b)

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%
027-023,10.000055,10.000055
029-002,10.000011,10.000011
088-023,9.999973,9.999973
112-014,10.000003,10.000003
080-017,5.000000,5.000000
...,...,...
069-017,5.000081,5.000081
077-008,9.999997,9.999997
084-011,10.000024,10.000024
123-005,10.000088,10.000088


# 推薦模式測試觀察重點 "調幅(op_opt-op)" 以及三項條件是否 "足夠趨近"

In [30]:
f = AllSystem(config)
idx = np.random.choice(allidx,size=1,replace=False,p=None)
print(icg_input.loc[idx,'Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'])
print(icg_input.loc[idx,'Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'])
print(icg_input.loc[idx,'Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'])
print(icg_input.loc[idx,'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'])

026-017    80.0
Name: Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%, dtype: float64
026-017    920.0
Name: Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw, dtype: float64
026-017    5.0
Name: Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw, dtype: float64
026-017    1.0
Name: Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr, dtype: float64


In [31]:
# 試算
c620_wt_estimate,c620_op_estimate,c660_wt_estimate,c660_op_estimate,c670_wt_estimate,c670_op_estimate = f.inference(icg_input.loc[idx],c620_feed.loc[idx],t651_feed.loc[idx])
# 推薦
c620_wt_optimal,c620_op_optimal,c660_wt_optimal,c660_op_optimal,c670_wt_optimal,c670_op_optimal,bz_error,nainbz_error,tol_error = f.recommend(icg_input.loc[idx],c620_feed.loc[idx],t651_feed.loc[idx],
                                                                            only_tune_temp=True)

[I 2021-05-04 04:51:55,106] A new study created in memory with name: no-name-e57360f0-cb54-4056-b2fc-130ef1e3f839


系統已經自動設定推薦值[70,980,10]


Congratulations Early Success find optimal op
bzinside_loss: 0.02535003720103321
nainbz_loss: 0.03265741632228884
tol_loss: 0.007925394671806352
distrate_loss: 0


# 確認三項條件有滿足

In [32]:
bz_error,nainbz_error,tol_error

(0.02535003720103321, 0.03265741632228884, 0.007925394671806352)

In [33]:
a = icg_input.loc[idx,['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]
b = c620_wt_optimal.loc[idx,['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]
c = pd.DataFrame(index=idx)
c['推薦值'] = 70
res = a.join(b).join(c)
res.columns = ['原始值','優化搜索後值','完美值']
res

,原始值,優化搜索後值,完美值
026-017,80.0,71.774503,70


In [34]:
a = icg_input.loc[idx,['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]
b = c660_wt_optimal.loc[idx].filter(regex='Side').filter(regex='wt%').iloc[:,[1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] ].sum(axis=1)*10000
b.name = 'c660_wt NA in NA in Benzene_ppmw'
c = pd.DataFrame(index=idx)
c['推薦值'] = 980
res = a.join(b).join(c)
res.columns = ['原始值','優化搜索後值','完美值']
res

,原始值,優化搜索後值,完美值
026-017,920.0,947.995732,980


In [35]:
a = icg_input.loc[idx,['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]
b = c660_wt_optimal.loc[idx,['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']]*10000
c = pd.DataFrame(index=idx)
c['推薦值'] = 10
res = a.join(b).join(c)
res.columns = ['原始值','優化搜索後值','完美值']
res

,原始值,優化搜索後值,完美值
026-017,5.0,10.079254,10


# 確認操作條件有足夠大的調幅,然後因為bzinside是往下降所以溫度調幅是往上升

In [36]:
(c620_op_optimal - c620_op_estimate).iloc[:,-2:]

,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
026-017,0.520956,0.511293


In [37]:
(c660_op_optimal - c660_op_estimate).iloc[:,-2:]

,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
026-017,-0.006032,0.285159


In [38]:
(c670_op_optimal - c670_op_estimate).iloc[:,-2:]

,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
026-017,-0.033859,0.042145


# 保存樣本

In [39]:
print(idx)

['026-017']


In [40]:
demo = {
      # input
      'icg_input':icg_input.loc[idx],
      'c620_feed':c620_feed.loc[idx],
      't651_feed':t651_feed.loc[idx],
      # output
      'c620_op':c620_op.loc[idx],
      'c620_wt':c620_wt.loc[idx],
      'c660_op':c660_op.loc[idx],
      'c660_wt':c660_wt.loc[idx],
      'c670_op':c670_op.loc[idx],
      'c670_wt':c670_wt.loc[idx],
      }

In [41]:
import joblib
joblib.dump(demo,"/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl")

['/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl']

# 保存模組

In [42]:
joblib.dump(f,"/content/drive/MyDrive/台塑輕油案子/data/c620/model/allsystem.pkl")

['/content/drive/MyDrive/台塑輕油案子/data/c620/model/allsystem.pkl']